In [2]:
import pandas as pd

In [18]:
input_shape = (1, 11)
input_shape

(1, 11)

In [3]:
a = pd.Series([1,2,3,4])

In [16]:
print(a.values.reshape(2,2))

[[1 2]
 [3 4]]


In [12]:
b = pd.DataFrame(a)

In [15]:
print(type(b.values))

<class 'numpy.ndarray'>


***

In [ ]:
"""
#TODO
Get a word_dict to convert all of the sequences to using the same pattern.
So that the conversion back to data gets easier if necessary.
This cell is the starter code to convert 'url', 'description' and 'tweets_dna'
to a consistent sequence of numbers.
"""

t  = Tokenizer()
fit_text = 'CTCCCAACACTCCACCCACCAAAAATCATACAATATTATAAACCAT'
t.fit_on_texts(fit_text)
test_text = 'CTCCCAACACTCCACCCACCAAAAATCATACAATATTATAAACCAT'
sequences = t.texts_to_sequences(test_text)

print("word_index : ",t.word_index)
print("sequences : ",sequences,'\n')